In [ ]:
import scipy
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
data = np.load("all_features.npy")

In [ ]:
print(data.shape)
labels_ids = [[1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3],
        [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1],
        [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]]

labels = np.zeros(data.shape[:-3])
for session in range(data.shape[0]):
    for video in range(data.shape[2]):
        labels[session,:,video] = labels_ids[session][video]

In [ ]:
def train_test(features, labels, test_subjects=[0,1,2]):
    test_data = features[:, test_subjects]
    test_labels = labels[:, test_subjects]

    train_data = np.delete(features, test_subjects, axis=1)
    train_labels = np.delete(labels, test_subjects, axis=1)

    # train_data = np.reshape(train_data, (-1, train_data.shape[-3],train_data.shape[-2], train_data.shape[-1]))
    print(train_data.shape)
    train_data = np.reshape(train_data, (-1, train_data.shape[-3]*train_data.shape[-2]* train_data.shape[-1]))
    train_labels = np.reshape(train_labels, (-1))

    # test_data = np.reshape(test_data, (-1, train_data.shape[-3],train_data.shape[-2],train_data.shape[-1]))

    test_data = np.reshape(test_data, (-1, test_data.shape[-3]*test_data.shape[-2]*test_data.shape[-1]))
    test_labels = np.reshape(test_labels, (-1))

    return train_data, train_labels, test_data, test_labels

In [ ]:
train_data, train_labels, test_data, test_labels = train_test(data, labels)

# normalize data
# train_data = (train_data - np.mean(train_data, axis=0)) / np.std(train_data, axis=0)
# test_data = (test_data - np.mean(train_data, axis=0)) / np.std(train_data, axis=0)

df1 = pd.DataFrame(train_data)
df2 = pd.DataFrame(test_data)
df = pd.concat([df1, df2])
df1['label'] = np.concatenate([train_labels])
df2['label'] = np.concatenate([test_labels])

df['label'] = np.concatenate((train_labels, test_labels))

df

In [ ]:
# KORELACIONA ANALIZA SA LABELAMA

# corr_analysis=abs(df.corr()['label']).sort_values(ascending=False)
# corr_analysis = corr_analysis[corr_analysis.index!='label']
# corr_analysis


In [ ]:
print(f"Shapes: {train_data.shape}, {train_labels.shape}, {test_data.shape}, {test_labels.shape}")

In [ ]:
knn = KNeighborsClassifier(5)
knn.fit(train_data,train_labels)
predicted_data = knn.predict(test_data)
acc = accuracy_score(test_labels,predicted_data)
acc

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

print(torch.cuda.is_available())
# clear cuda cache
torch.cuda.empty_cache()

In [ ]:
hidden_size = 500
net = nn.Sequential(
    nn.Linear(10*62*4, hidden_size),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(hidden_size, 4),
    nn.Sigmoid()
).to('cuda')

optimizer = torch.optim.Adam(net.parameters(), lr=0.00001)
loss_fn = nn.CrossEntropyLoss()

train_data = torch.tensor(train_data, dtype=torch.float32).to('cuda')
train_labels = torch.tensor(train_labels, dtype=torch.long).to('cuda')
test_data = torch.tensor(test_data, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)
# convert labels to one-hot encoding
train_labels = F.one_hot(train_labels, num_classes=4).float()
test_labels = F.one_hot(test_labels, num_classes=4).float()



In [ ]:
net.train()
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)
net.apply(init_weights)
for epoch in range(100000):
    # clear cuda cache
    torch.cuda.empty_cache()
    optimizer.zero_grad()
    out = net(train_data)
    # print(f"out: {out.shape}, train_labels: {train_labels.shape}, train_data: {train_data.shape}")
    loss = loss_fn(out, train_labels)
    
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        acc = accuracy_score(torch.argmax(train_labels, dim=1).cpu(), torch.argmax(out, dim=1).cpu())
        # test the model
        with torch.no_grad():
            out = net(test_data.to('cuda'))
            predicted_labels = torch.argmax(out, dim=1)
            test_non_one_hot = torch.argmax(test_labels, dim=1)
            test_acc = accuracy_score(test_non_one_hot, predicted_labels.cpu())
        print(f'Epoch {epoch}, loss {loss.item()} train_accuracy {acc} test_accuracy {test_acc}')


In [ ]:
# generate confusion matrix
net.eval()
with torch.no_grad():
    out = net(test_data.to('cuda'))
    predicted_labels = torch.argmax(out, dim=1)
    test_non_one_hot = torch.argmax(test_labels, dim=1)
    test_acc = accuracy_score(test_non_one_hot, predicted_labels.cpu())
    cm = confusion_matrix(test_non_one_hot, predicted_labels.cpu())
    print(f"Test accuracy: {test_acc}")
    print(cm)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['0','1','2','3'])
    disp.plot()
    plt.show()